<a href="https://colab.research.google.com/github/Chainka-Mos/ML_CW_FraudScoring/blob/master/%D0%A1%D0%A3%D0%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import math

## **Задача №4. Поражение наблюдаемой цели с пристрелкой с помощью дальномера** 

Ввод условий задачи

In [0]:
#Это временный ввод, как лучше разберемся позже, пока так удобно обрабатывать
#Ввод координат Огневой позиции
X_op,Y_op,h_op,alpha_on = list(map(float,'53850,04500,150,1.00'.split(sep = ',')))
#Ввод координат 
X_knp,Y_knp,h_knp = list(map(float,'55782,04845,170'.split(sep = ',')))
#таблица поправок
D = [4,6,8]
delta_D = [120,150,220]
delta_dovorot = [-0.05,-0.09,-0.16]

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Контроль1
# функция прямой геодезической задачи
# X_nt,Y_nt - координаты точки отсчета
# alpha,D - угол и дальность на цель
def PGZ(X_nt,Y_nt,alpha,D):
    X_c = round(X_nt+D*math.cos(alpha*6*math.pi/180),0)
    Y_c = round(Y_nt+D*math.sin(alpha*6*math.pi/180),0)
    return X_c,Y_c
X_kt,Y_kt = PGZ(X_knp,Y_knp,alpha_on,3000)

#Рассчеты по Кравченко
def Kravchenko(X_op,Y_op,X_c,Y_c,toPrint = True,Kontrol_1 = False,BRK = False):
    #вводим таблицы
    Kravchenko1_4 = pd.read_csv('Kravchenko1-4.csv',sep = ';')
    Kravchenko5_8 = pd.read_csv('Kravchenko5-8.csv',sep = ';')
    #считаем разницы координат
    delta_X = X_c - X_op
    delta_Y = Y_c - Y_op
    #выясняем, кто больше, кто меньше
    MRK = min(delta_X,delta_Y)
    BRK = max(delta_X,delta_Y)
    K_n = abs(round(MRK/BRK,3))#округляем до трех знаков и берем модуль
    K_d = 0#объявляем переменную коэф-та дальности

    def infoKravchenko(Table,column):
        #cреди всех строчек таблицы Кравченко
        K_d = 0
        width = 0#ширина окна поиска значений
        while K_d==0:
            for index,row in Table.iterrows():
                #ищем совпадающий K_n или ближайший
                if row['K_n']==K_n*1000-width:
                    #запоминаем интересующие значения в нужном формате
                    K_d = row['K_d']/1000 + 1
                    alpha_t_c = row[column]+row['100-e']/100
            width += 1
        return K_d,alpha_t_c
        
    #блок отбора условий
    if MRK>0:
        if MRK == delta_X:
            if BRK<0:
                K_d,alpha_t_c = infoKravchenko(Kravchenko1_4,"+X/-Y")
            if BRK>0:
                K_d,alpha_t_c = infoKravchenko(Kravchenko5_8,"+X/-Y")
        if MRK == delta_Y:
            if BRK>0:
                K_d,alpha_t_c = infoKravchenko(Kravchenko1_4,"+Y/+X")
            if BRK<0:
                K_d,alpha_t_c = infoKravchenko(Kravchenko5_8,"+Y/-X")
    if MRK<0:
        if MRK == delta_X:
            if BRK<0:
                K_d,alpha_t_c = infoKravchenko(Kravchenko5_8,"-X/-Y")
            if BRK>0:
                K_d,alpha_t_c = infoKravchenko(Kravchenko1_4,"-X/+Y")
        if MRK == delta_Y:
            if BRK>0:
                K_d,alpha_t_c = infoKravchenko(Kravchenko5_8,"-Y/+X")
            if BRK<0:
                K_d,alpha_t_c = infoKravchenko(Kravchenko1_4,"-Y/-X")
    if toPrint:
        log  = {'1.X_ц':X_c,'2.X_оп':X_op,'3.ΔX =X_ц-X_оп':delta_X,'4.Y_ц':Y_c,'5.Y_оп':Y_op,
                            '6.ΔY =Y_ц-Y_оп':delta_Y,'7.К_н = МРК:БРК':K_n,'8.a_т_ц':alpha_t_c,'9.a_он':alpha_on,
                             '10.∂_т_ц = a_т_ц -a_он':alpha_t_c-alpha_on,'11.K_д':K_d,'12.Д_т^ц = БРК*К_d': round(K_d*BRK,0)}
        logs = pd.Series(log)
        print(logs)
    if BRK:
        return K_d,alpha_t_c,BRK
    if Kontrol_1:
        Doklad = "Доклад командира батареи Дон,я Волга. Контроль-1 дальность _{}_ ОН _{}_".format(K_d*BRK,alpha_t_c-alpha_on)
        print('-'*(len(Doklad)))
        print(Doklad)
    return K_d,alpha_t_c


In [0]:
#Доклад разведчика 
alpha_c,M_c = 59.28,-0.07
#доклад дальномерщика
D_c,G_c = 3550,70
#дополнительные cведения(фронт цели)
F_c = 0.30
#определение координат цели, решая ПГЗ от КНП
X_c,Y_c = PGZ(X_knp,Y_knp,alpha_c,D_c)
print("Координаты Цели X_ц ={}, Y_ц = {}".format(X_c,Y_c))
#раccчет выcоты цели
h_c = round(h_knp + (M_c*100)*0.001*D_c*1.05,0)
print('Выcота цели = {} метров'.format(h_c))

#ГРП
delta_D_i = +146
delta_dovorot_i = -0.08
Pricel = 319

#cнова cкажем привет таблице Кравченко
print('-'*100)
K_d,alpha_t_c,BRK = Kravchenko(X_op,Y_op,X_c,Y_c,Kontrol_1 = True,BRK  =True)
print('-'*100)
#вывод предварительной информации для построения таблицы

K_u = round(D_c/(BRK*K_d),1)
PS = round((alpha_t_c - alpha_c+60)%60,2)
ShU = round(PS/(0.01*BRK*K_d),2)
delta_X_t = 13
PreTableDict = {'Д_к':D_c,'a_ц':alpha_c,'K_у =Д_к/Д_т^ц ':K_u,'Пс = a_т - а_ц':PS,'Шу = Пc/(0,01*Д_т^ц)':ShU,'ΔX_тыс':delta_X_t}
PreTableInfo = pd.Series(PreTableDict)
print(PreTableInfo)
print('-'*100)    
# def FirstSteptype4(Pr,Ur,dovorot,alpha,Dalnost):
    


NameError: ignored

In [0]:
#Доклад разведчика 
alpha_c,M_c = 59.28,-0.07
#доклад дальномерщика
D_c,G_c = 3550,70
#дополнительные cведения(фронт цели)
F_c = 0.30
#определение координат цели, решая ПГЗ от КНП
X_c,Y_c = PGZ(X_knp,Y_knp,alpha_c,D_c)
print("Координаты Цели X_ц ={}, Y_ц = {}".format(X_c,Y_c))
#раccчет выcоты цели
h_c = round(h_knp + (M_c*100)*0.001*D_c*1.05,0)
print('Выcота цели = {} метров'.format(h_c))

#ГРП
delta_D_i = +146
delta_dovorot_i = -0.08
pricel = 319
# рассчет уровня

#cнова cкажем привет таблице Кравченко
print('-'*100)
K_d,alpha_t_c,BRK = Kravchenko(X_op,Y_op,X_c,Y_c,Kontrol_1 = True,BRK  =True)
print('-'*100)
#вывод предварительной информации для построения таблицы

K_u = round(D_c/(BRK*K_d),1)
PS = round((alpha_t_c - alpha_c+60)%60,2)
ShU = round(PS/(0.01*BRK*K_d),2)
delta_X_t = 13
PreTableDict = {'Д_к':D_c,'a_ц':alpha_c,'K_у =Д_к/Д_т^ц ':K_u,'Пс = a_т - а_ц':PS,'Шу = Пc/(0,01*Д_т^ц)':ShU,'ΔX_тыс':delta_X_t}
PreTableInfo = pd.Series(PreTableDict)
print(PreTableInfo)
print('-'*100)    

def Routine(alpha_r,alpha_c,Dalnost_r,Dalnost_c,X_tis,K_u,ShU,pravolevo):
    #в рамках одной задачи имеет смысл менять тут только alpha_r b Dalnost_r
    #alpha_r - разрывов,Dalnost_r - аналогично,X_tis - X_тыс,pravolevo = {0,1}положение ОП 0 - справа, 1, слева
    #функция рассчета прицела и доворота 
    delta_D = Dalnost_r - Dalnost_c
    delta_pricel = round(-delta_D/X_tis,0)
    delta_alpha = (alpha_r - alpha_c+60)%60
    if pravolevo == 0:
        if delta_D>0:
            delta_dovorot = round((-delta_alpha*100*K_u-delta_D*ShU)/100,2)
        if delta_D<0:
            delta_dovorot = round((-delta_alpha*100*K_u+delta_D*ShU)/100,2)
    if pravolevo == 1:
        if delta_D<0:
            delta_dovorot = round((-delta_alpha*100*K_u-delta_D*ShU)/100,2)
        if delta_D>0:
            delta_dovorot = round((-delta_alpha*100*K_u+delta_D*ShU)/100,2)
    return delta_pricel,delta_dovorot
    
# def FirstSteptype4(Pricel,Ur,dovorot,alpha,Dalnost):
    #отсчитывать количество снарядов
    
#     новый прицел и доворот
alpha_r = 0.22
Dalnost_r = 3310
Dalnost_c = D_c
X_tis = delta_X_t
delta_Pricel,delta_dovorot = Routine(alpha_r,alpha_c,Dalnost_r,Dalnost_c,X_tis,K_u,ShU,pravolevo=0)
print('Прицел меняется на = ',delta_Pricel,'Доворот = ',delta_dovorot)
#в рамках одной задачи имеет смысл менять тут только alpha_r b Dalnost_r

Координаты Цели X_ц =59322.0, Y_ц = 4578.0
Высота цели = 144.0 метров
1.X_ц                     59322.000
2.X_оп                    53850.000
3.ΔX =X_ц-X_оп             5472.000
4.Y_ц                      4578.000
5.Y_оп                     4500.000
6.ΔY =Y_ц-Y_оп               78.000
7.К_н = МРК:БРК               0.014
8.a_т_ц                       0.120
9.a_он                        1.000
10.∂_т_ц = a_т_ц -a_он       -0.880
11.K_д                        1.000
12.Д_т^ц = БРК*К_d         5472.000
dtype: float64
------------------------------------------------------------------------------
Доклад командира батареи Дон,я Волга. Контроль-1 дальность _5472.0_ ОН _-0.88_


(1.0, 0.12)

## **Задача № 1 - Тема № 5, 7. Расчет поправок на отклонение условий стрельбы, построение графика рассчитанных поправок (ГРП), расчет установок для стрельбы аналитическим способом и подготовка команды на поражение ненаблюдаемой цели без пристрелки.** 

In [0]:
#считывание таблиц стрельбы по зарядам
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sostavliaushie = pd.read_csv('/content/drive/My Drive/СУО_ДАННЫЕ/Разложение на составляющие СУО - Лист2.csv')
full_ch = pd.read_csv('/content/drive/My Drive/СУО_ДАННЫЕ/Полный.csv',sep = ';',header = None)
reduced_ch = pd.read_csv('/content/drive/My Drive/СУО_ДАННЫЕ/Уменьшенный.csv',sep = ';',header = None)
first_ch = pd.read_csv('/content/drive/My Drive/СУО_ДАННЫЕ/Первый.csv',sep = ';',header = None)
second_ch = pd.read_csv('/content/drive/My Drive/СУО_ДАННЫЕ/Второй.csv',sep = ';',header = None)
third_ch = pd.read_csv('/content/drive/My Drive/СУО_ДАННЫЕ/Третий.csv',sep = ';',header = None)
fourth_ch = pd.read_csv('/content/drive/My Drive/СУО_ДАННЫЕ/Четвертый.csv',sep = ';',header = None)

In [81]:
#ввод бюллетени
#вызов функции в самом низу!
#прописывайте высоты без нулей в начале!
meteobulluten = ['01415',{'150':'50354','252':'514603','452':'514604','852':'524604','1253':'524705','1653':'524906','2054':'535008','2454':'545108','3003':'555209','4054':'555310'}]#метеобюллетень.вводить до последней пары!
#---------------------------------------------------------------------------------------------------------------
def Popravki(h_op,alpha_on,zaryad,T_z,deltaV_0_sum,deltaV_0_osn,D_op,meteobulluten):
#1.Определение отклонений баллист.условий от табличных
    delta_V_0 = deltaV_0_sum+deltaV_0_osn
    delta_T_z = T_z-15
    print('ΔV_0_сум : {}'.format(round(delta_V_0,2)))
    print('ΔТ_з : ',delta_T_z)
#2.Определение отклонений метеоролог. условий стрельбы от табличных
    try:
      if int(list(meteobulluten[1].values())[0][0]) >0:#проверяем, является значения дельта высоты места цели(delta H_mc) отрицательным
        delta_h_op = -(int(list(meteobulluten[1].values())[0][0])-5)*10-(int(list(meteobulluten[1].values())[0][1:3]))+(int(list(meteobulluten[1].keys())[0])-h_op)/10
      else:
        delta_h_op = int(list(meteobulluten[1].values())[0][1:3])+(int(list(meteobulluten[1].keys())[0])-h_op)/10
    except SyntaxError:#если вылезет ошибка, что значение температуры это 03 градуса, например, то обработаем ошибку
      if int(list(meteobulluten[1].values())[0][0]) > 0:#проверяем, является значения дельта высоты места цели(delta H_mc) отрицательным
        delta_h_op = -(int(list(meteobulluten[1].values())[0][0])-5)*10-int(list(meteobulluten[1].values())[0][2])+(int(list(meteobulluten[1].keys())[0])-h_op)/10
      else:
        delta_h_op = int(list(meteobulluten[1].values())[0][2])+(int(list(meteobulluten[1].keys())[0])-h_op)/10
    print('Дельта h_op: ',delta_h_op)
#---------------------------------------------------------------------------------------------------------------
    Y_entry_height = []
    delta_D_W_x = []
    delta_D_H_op = []
    delta_D_T_v0 = []
    delta_D_T_z = []
    delta_D_v0_sum = []
    delta_W_z = []
    Z = []
    def charge(charge_list, distance, counter_Y_bul=0):#создаем функцию, чтобы не пихать одинаковые циклы обработки для каждого заряда
      for j in charge_list[0]:
        counter_Y_bul += 1
        if j == distance:
          return(charge_list.iloc[[counter_Y_bul-1],:])

    for i in D_op:#Поиск высоты входа в бюллетень
      if zaryad == -1:
        Y_entry_height.append(int(charge(full_ch,i)[12]))
        delta_D_W_x.append(-0.1*int(charge(full_ch,i)[7]))
        delta_D_H_op.append(0.1*int(charge(full_ch,i)[8]))
        delta_D_T_v0.append(-0.1*int(charge(full_ch,i)[9]))
        delta_D_T_z.append(-0.1*int(charge(full_ch,i)[10]))
        delta_D_v0_sum.append(-int(charge(full_ch,i)[11]))
        delta_W_z.append(0.1*-int(charge(full_ch,i)[6]))
        Z.append(-int(charge(full_ch,i)[5]))
      elif zaryad == 0:
        Y_entry_height.append(int(charge(reduced_ch,i)[12]))
        delta_D_W_x.append(-0.1*int(charge(reduced_ch,i)[7]))
        delta_D_H_op.append(0.1*int(charge(reduced_ch,i)[8]))
        delta_D_T_v0.append(-0.1*int(charge(reduced_ch,i)[9]))
        delta_D_T_z.append(-0.1*int(charge(reduced_ch,i)[10]))
        delta_D_v0_sum.append(-int(charge(reduced_ch,i)[11]))
        delta_W_z.append(0.1*-int(charge(reduced_ch,i)[6]))
        Z.append(-int(charge(reduced_ch,i)[5]))
      elif zaryad == 1:
        Y_entry_height.append(int(charge(first_ch,i)[12]))
        delta_D_W_x.append(-0.1*int(charge(first_ch,i)[7]))
        delta_D_H_op.append(0.1*int(charge(first_ch,i)[8]))
        delta_D_T_v0.append(-0.1*int(charge(first_ch,i)[9]))
        delta_D_T_z.append(-0.1*int(charge(first_ch,i)[10]))
        delta_D_v0_sum.append(-int(charge(first_ch,i)[11]))
        delta_W_z.append(0.1*-int(charge(first_ch,i)[6]))
        Z.append(-int(charge(first_ch,i)[5]))
      elif zaryad == 2:
        Y_entry_height.append(int(charge(second_ch,i)[12]))
        delta_D_W_x.append(-0.1*int(charge(second_ch,i)[7]))
        delta_D_H_op.append(0.1*int(charge(second_ch,i)[8]))
        delta_D_T_v0.append(-0.1*int(charge(second_ch,i)[9]))
        delta_D_T_z.append(-0.1*int(charge(second_ch,i)[10]))
        delta_D_v0_sum.append(-int(charge(second_ch,i)[11]))
        delta_W_z.append(0.1*-int(charge(second_ch,i)[6]))
        Z.append(-int(charge(second_ch,i)[5]))
      elif zaryad == 3:
        Y_entry_height.append(int(charge(third_ch,i)[12]))
        delta_D_W_x.append(-0.1*int(charge(third_ch,i)[7]))
        delta_D_H_op.append(0.1*int(charge(third_ch,i)[8]))
        delta_D_T_v0.append(-0.1*int(charge(third_ch,i)[9]))
        delta_D_T_z.append(-0.1*int(charge(third_ch,i)[10]))
        delta_D_v0_sum.append(-int(charge(third_ch,i)[11]))
        delta_W_z.append(0.1*-int(charge(third_ch,i)[6]))
        Z.append(-int(charge(third_ch,i)[5]))
      else:
        Y_entry_height.append(int(charge(fourth_ch,i)[12]))
        delta_D_W_x.append(-0.1*int(charge(fourth_ch,i)[7]))
        delta_D_H_op.append(0.1*int(charge(fourth_ch,i)[8]))
        delta_D_T_v0.append(-0.1*int(charge(fourth_ch,i)[9]))
        delta_D_T_z.append(-0.1*int(charge(fourth_ch,i)[10]))
        delta_D_v0_sum.append(-int(charge(fourth_ch,i)[11]))
        delta_W_z.append(0.1*-int(charge(fourth_ch,i)[6]))
        Z.append(-int(charge(fourth_ch,i)[5]))
    print('Y_бюлл',Y_entry_height)
#--------------------------------------------------------------------------------------------
    bulletin_group = []#Поиск группы бюллетеня
    for j in Y_entry_height:
      bottom  = []
      upper = []
      for i in meteobulluten[1].keys():
        if int(i)<j:
          bottom.append(int(i))
        elif int(i)>j:
          upper.append(int(i))
        elif int(i) == j:
          bottom = upper = [int(i)]
          break
      bulletin_group.append([bottom[-1],upper[0]])
#-----------------------------------------------------------------------------------------------------------
    delta_T_v0 = []#достаем значения из группы бюллетеней
    alpha_w = []
    W = []
    counter_bulletin = 0
    def displacement(Y_entry,initial,bulletin_height):
      if np.sign(initial[0]) != np.sign(initial[1]):
        print('OOOps, значения температур даны с разными знаками, я не знаю, как обработать эту ошибку. Крепись.')
      #print(abs(initial[1])-abs(initial[0]))
      #print(Y_entry-bulletin_height[0])
      #print(bulletin_height[1]-bulletin_height[0])
      return(initial[0]+np.sign(initial[0])*(Y_entry-bulletin_height[0])*(abs(initial[1])-abs(initial[0]))/(bulletin_height[1]-bulletin_height[0]))
    for i in bulletin_group:
      if  int(meteobulluten[1].get(str(i[0]))[0])>0:#определяем дельта T_v0
        Tv_1 = -int(meteobulluten[1].get(str(i[0]))[1])-(int(meteobulluten[1].get(str(i[0]))[0])-5)*10
      else:
        Tv_1 = int(meteobulluten[1].get(str(i[0]))[1])
      if int(meteobulluten[1].get(str(i[1]))[0])>0:
        Tv_2 = -int(meteobulluten[1].get(str(i[1]))[1])-(int(meteobulluten[1].get(str(i[1]))[0])-5)*10
      else:
        Tv_2 = int(meteobulluten[1].get(str(i[1]))[1])
      delta_T_v0.append(displacement(Y_entry_height[counter_bulletin],[Tv_1,Tv_2],[i[0],i[1]]))
      
#---------------------------------------------------------------------------------------------------------------
      if  int(meteobulluten[1].get(str(i[0]))[2])==0:#определяем alpha_w
        al_1 = int(meteobulluten[1].get(str(i[0]))[3])
      else:
        al_1 = int(meteobulluten[1].get(str(i[0]))[2:4])
      if  int(meteobulluten[1].get(str(i[1]))[2])==0:#определяем alpha_w
        al_2 = int(meteobulluten[1].get(str(i[1]))[3])
      else:
        al_2 = int(meteobulluten[1].get(str(i[1]))[2:4])   
      alpha_w.append(displacement(Y_entry_height[counter_bulletin],[al_1,al_2],[i[0],i[1]]))
#---------------------------------------------------------------------------------------------------------------
      if  int(meteobulluten[1].get(str(i[0]))[4])==0:#определяем W
        w_1 = int(meteobulluten[1].get(str(i[0]))[5])
      else:
        w_1 = int(meteobulluten[1].get(str(i[0]))[5:])
      if  int(meteobulluten[1].get(str(i[1]))[4])==0:#определяем W
        w_2 = int(meteobulluten[1].get(str(i[1]))[5])
      else:
        w_2 = int(meteobulluten[1].get(str(i[1]))[4:])   
      W.append(displacement(Y_entry_height[counter_bulletin],[w_1,w_2],[i[0],i[1]]))
      counter_bulletin+=1
#----------------------------------------------------------------------------------------------------------------
    print('Сформируйте группу бюллетеня из представленных коэффициентов')
    print('ΔТv0: ', delta_T_v0)
    print('aw :',alpha_w)
    print('W',W)
    Ax = [] #рассчет Ax
    for i in alpha_w:
      if alpha_on>i:
        Ax.append(round(alpha_on-i))
      else:
        Ax.append(round(alpha_on+60-i))
#-----------------------------------------------------------------------------------------------------------------
    def sostavliaushie_fun(point):#разложение ветра на составляющие
        for key,value in sostavliaushie.iloc[:,:4].iteritems():
            for i in value:
              counter[0] +=1
              if counter[0] == 15:
                counter[0]= 0
                counter[1]+=1
              if i == point:
                result = counter[0] - counter[1]
        return(result)

    Wx_Wz = []
    for i in range(len(Ax)):
        counter = [0,0]
        result = sostavliaushie_fun(Ax[i])
        if Ax[i] in list(sostavliaushie.iloc[:,0]):
            signs = [-1,+1]
        elif Ax[i] in list(sostavliaushie.iloc[:,1]):
            signs = [+1,+1]
        elif Ax[i] in list(sostavliaushie.iloc[:,2]):
            signs = [+1,-1]
        elif Ax[i] in list(sostavliaushie.iloc[:,3]):
            signs = [-1,-1]
        Wx_Wz.append([int(np.array(str(sostavliaushie.iloc[result-1,3+int(round(W[i]))]).split('/'))[0])*signs[0],int(np.array(str(sostavliaushie.iloc[result-1,3+int(round(W[i]))]).split('/'))[1])*signs[1]])
#-----------------------------------------------------------------------------------------------------------
    print('Ax',Ax)
    print('Wx_Wz (каждый внутренний массив содержит сначала Wx, а потом Wz для дальности): ',Wx_Wz)

#Рассчет поправок на отклонение условий стрельбы от табличных
    for i in range(len(Wx_Wz)):
        print('--------------------------------------------------------------------------------------------')
        print('Для Д_оп {}'.format(Y_entry_height[i]))
        print('ΔДWx: ',delta_D_W_x[i],Wx_Wz[i][0],delta_D_W_x[i]*Wx_Wz[i][0])
        print('ΔДHop: ',delta_D_H_op[i],delta_h_op,delta_D_H_op[i]*delta_h_op)
        print('ΔДTv0: ',delta_D_T_v0[i],delta_T_v0[i],delta_D_T_v0[i]*delta_T_v0[i])
        print('ΔДTz: ',delta_D_T_z[i],delta_T_z,delta_D_T_z[i]*delta_T_z)
        print('ΔДTV0_sum: ',delta_D_v0_sum[i],delta_V_0,delta_D_v0_sum[i]*delta_V_0)
        print('ΔД_SUM: ',delta_D_W_x[i]*Wx_Wz[i][0]+delta_D_H_op[i]*delta_h_op+delta_D_T_v0[i]*delta_T_v0[i]+delta_D_T_z[i]*delta_T_z+delta_D_v0_sum[i]*delta_V_0)
        print('Δ∂Wz (в малых делениях угломера!): ',Wx_Wz[i][1],delta_W_z[i],Wx_Wz[i][1]*delta_W_z[i])
        print('Z(в малых): ',Z[i])
        print('Δ∂_SUM(в малых): ',Z[i]+Wx_Wz[i][1]*delta_W_z[i])
        print('ДтГР',D_op[i]-(delta_D_W_x[i]*Wx_Wz[i][0]+delta_D_H_op[i]*delta_h_op+delta_D_T_v0[i]*delta_T_v0[i]+delta_D_T_z[i]*delta_T_z+delta_D_v0_sum[i]*delta_V_0))
#-----------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------
Popravki(180,47,-1,10,-1.2,-0.6,[10000,12000,14000],meteobulluten)#заряд -1 == полный, заряд 0 == уменьшенный, 1 и далее по списку
# по порядку: h_op,alpha_on,zaryad,T_z,deltaV_0_sum,deltaV_0_osn,D_op,meteobulluten



ΔV_0_сум : -1.8
ΔТ_з :  -5
Дельта h_op:  -6.0
Y_бюлл [1400, 2400, 4000]
Сформируйте группу бюллетеня из представленных коэффициентов
ΔТv0:  [-2.0, -3.865, -5.0]
aw : [47.735, 50.865, 52.948620361560415]
W [5.3675, 8.0, 9.948620361560419]
Ax [59.0, 56.0, 54.0]
Wx_Wz (каждый внутренний массив содержит сначала Wx, а потом Wz для дальности):  [[-5, -1], [-7, -3], [-8, -6]]
--------------------------------------------------------------------------------------------
Для Д_оп 1400
ΔДWx:  -19.8 -5 99.0
ΔДHop:  6.2 -6.0 -37.2
ΔДTv0:  -16.400000000000002 -2.0 32.800000000000004
ΔДTz:  -8.5 -5 42.5
ΔДTV0_sum:  -106 -1.7999999999999998 190.79999999999998
ΔД_SUM:  327.9
Δ∂Wz (в малых делениях угломера!):  -1 -1.2000000000000002 1.2000000000000002
Z(в малых):  -7
Δ∂_SUM(в малых):  -5.8
ДтГР 9672.1
--------------------------------------------------------------------------------------------
Для Д_оп 2400
ΔДWx:  -28.6 -7 200.20000000000002
ΔДHop:  7.7 -6.0 -46.2
ΔДTv0:  -21.5 -3.865 83.09750000000001
Δ

SyntaxError: ignored